## 1. Introduction
<p><img src="https://assets.datacamp.com/production/project_1197/img/google_play_store.png" alt="Google Play logo"></p>
<p>Mobile apps are everywhere. They are easy to create and can be very lucrative from the business standpoint. Specifically, Android is expanding as an operating system and has captured more than 74% of the total market<sup><a href="https://www.statista.com/statistics/272698/global-market-share-held-by-mobile-operating-systems-since-2009">[1]</a></sup>. </p>
<p>The Google Play Store apps data has enormous potential to facilitate data-driven decisions and insights for businesses. In this notebook, we will analyze the Android app market by comparing ~10k apps in Google Play across different categories. We will also use the user reviews to draw a qualitative comparision between the apps.</p>
<p>The dataset you will use here was scraped from Google Play Store in September 2018 and was published on <a href="https://www.kaggle.com/lava18/google-play-store-apps">Kaggle</a>. Here are the details: <br>
<br></p>
<div style="background-color: #efebe4; color: #05192d; text-align:left; vertical-align: middle; padding: 15px 25px 15px 25px; line-height: 1.6;">
    <div style="font-size:20px"><b>datasets/apps.csv</b></div>
This file contains all the details of the apps on Google Play. There are 9 features that describe a given app.
<ul>
    <li><b>App:</b> Name of the app</li>
    <li><b>Category:</b> Category of the app. Some examples are: ART_AND_DESIGN, FINANCE, COMICS, BEAUTY etc.</li>
    <li><b>Rating:</b> The current average rating (out of 5) of the app on Google Play</li>
    <li><b>Reviews:</b> Number of user reviews given on the app</li>
    <li><b>Size:</b> Size of the app in MB (megabytes)</li>
    <li><b>Installs:</b> Number of times the app was downloaded from Google Play</li>
    <li><b>Type:</b> Whether the app is paid or free</li>
    <li><b>Price:</b> Price of the app in US$</li>
    <li><b>Last Updated:</b> Date on which the app was last updated on Google Play </li>

</ul>
</div>
<div style="background-color: #efebe4; color: #05192d; text-align:left; vertical-align: middle; padding: 15px 25px 15px 25px; line-height: 1.6;">
    <div style="font-size:20px"><b>datasets/user_reviews.csv</b></div>
This file contains a random sample of 100 <i>[most helpful first](https://www.androidpolice.com/2019/01/21/google-play-stores-redesigned-ratings-and-reviews-section-lets-you-easily-filter-by-star-rating/)</i> user reviews for each app. The text in each review has been pre-processed and passed through a sentiment analyzer.
<ul>
    <li><b>App:</b> Name of the app on which the user review was provided. Matches the `App` column of the `apps.csv` file</li>
    <li><b>Review:</b> The pre-processed user review text</li>
    <li><b>Sentiment Category:</b> Sentiment category of the user review - Positive, Negative or Neutral</li>
    <li><b>Sentiment Score:</b> Sentiment score of the user review. It lies between [-1,1]. A higher score denotes a more positive sentiment.</li>

</ul>
</div>
<p>From here on, it will be your task to explore and manipulate the data until you are able to answer the three questions described in the instructions panel.<br></p>

In [67]:
# Use this cell to begin your analysis, and add as many as you would like!
import pandas as pd
import numpy as np 

In [68]:
apps = pd.read_csv('datasets/apps.csv')
apps.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Last Updated
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19.0,"10,000+",Free,0.0,"January 7, 2018"
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14.0,"500,000+",Free,0.0,"January 15, 2018"
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7,"5,000,000+",Free,0.0,"August 1, 2018"
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25.0,"50,000,000+",Free,0.0,"June 8, 2018"
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8,"100,000+",Free,0.0,"June 20, 2018"


In [69]:
apps.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9659 entries, 0 to 9658
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   App           9659 non-null   object 
 1   Category      9659 non-null   object 
 2   Rating        8196 non-null   float64
 3   Reviews       9659 non-null   int64  
 4   Size          8432 non-null   float64
 5   Installs      9659 non-null   object 
 6   Type          9659 non-null   object 
 7   Price         9659 non-null   float64
 8   Last Updated  9659 non-null   object 
dtypes: float64(3), int64(1), object(5)
memory usage: 679.3+ KB


In [70]:
apps.describe()

,Rating,Reviews,Size,Price
count,8196.000000,9.659000e+03,8432.000000,9659.000000
mean,4.173243,2.165926e+05,20.395327,1.099299
std,0.536625,1.831320e+06,21.827509,16.852152
min,1.000000,0.000000e+00,0.000000,0.000000
25%,4.000000,2.500000e+01,4.600000,0.000000
50%,4.300000,9.670000e+02,12.000000,0.000000
75%,4.500000,2.940100e+04,28.000000,0.000000
max,5.000000,7.815831e+07,100.000000,400.000000


In [71]:
apps.shape

(9659, 9)

In [72]:
apps.isna().sum()

App                0
Category           0
Rating          1463
Reviews            0
Size            1227
Installs           0
Type               0
Price              0
Last Updated       0
dtype: int64

In [73]:
apps['Installs'] = apps['Installs'].astype('str').str.replace('+', '')

In [74]:
apps['Installs'] = apps['Installs'].astype('str').str.replace(',', '')

In [75]:
apps['Installs'] = apps['Installs'].astype('int')

In [76]:
num_of_apps = apps.Category.value_counts()
print(num_of_apps)

FAMILY                 1832
GAME                    959
TOOLS                   827
BUSINESS                420
MEDICAL                 395
PERSONALIZATION         376
PRODUCTIVITY            374
LIFESTYLE               369
FINANCE                 345
SPORTS                  325
COMMUNICATION           315
HEALTH_AND_FITNESS      288
PHOTOGRAPHY             281
NEWS_AND_MAGAZINES      254
SOCIAL                  239
BOOKS_AND_REFERENCE     222
TRAVEL_AND_LOCAL        219
SHOPPING                202
DATING                  171
VIDEO_PLAYERS           163
MAPS_AND_NAVIGATION     131
EDUCATION               119
FOOD_AND_DRINK          112
ENTERTAINMENT           102
AUTO_AND_VEHICLES        85
LIBRARIES_AND_DEMO       84
WEATHER                  79
HOUSE_AND_HOME           74
EVENTS                   64
ART_AND_DESIGN           64
PARENTING                60
COMICS                   56
BEAUTY                   53
Name: Category, dtype: int64


In [77]:
average_price = apps.groupby('Category')['Price'].mean()

In [78]:
average_ratings = apps.groupby('Category')['Rating'].mean()

In [79]:
app_category_info = pd.concat([num_of_apps, average_price, average_ratings], axis = 1)

In [80]:
type(app_category_info)

pandas.core.frame.DataFrame

In [81]:
app_category_info.head()

,Category,Price,Rating
FAMILY,1832,1.309967,4.179664
GAME,959,0.296465,4.247368
TOOLS,827,0.323156,4.039554
BUSINESS,420,0.417357,4.098479
MEDICAL,395,2.520759,4.166552


In [82]:
app_category_info = app_category_info.reset_index()

In [83]:
app_category_info.columns = ['Category', 'Number of apps', 'Average price', 'Average rating']

In [84]:
app_category_info

,Category,Number of apps,Average price,Average rating
0,FAMILY,1832,1.309967,4.179664
1,GAME,959,0.296465,4.247368
2,TOOLS,827,0.323156,4.039554
3,BUSINESS,420,0.417357,4.098479
4,MEDICAL,395,2.520759,4.166552
5,PERSONALIZATION,376,0.400213,4.332215
6,PRODUCTIVITY,374,0.670936,4.183389
7,LIFESTYLE,369,6.398022,4.093355
8,FINANCE,345,8.408203,4.115563
9,SPORTS,325,0.307692,4.216154


In [85]:
df2 = pd.read_csv('datasets/user_reviews.csv')

In [86]:
df2.head()

,App,Review,Sentiment Category,Sentiment Score
0,10 Best Foods for You,I like eat delicious food. That's I'm cooking ...,Positive,1.00
1,10 Best Foods for You,This help eating healthy exercise regular basis,Positive,0.25
2,10 Best Foods for You,NaN,NaN,NaN
3,10 Best Foods for You,Works great especially going grocery store,Positive,0.40
4,10 Best Foods for You,Best idea us,Positive,1.00


In [87]:
testing = pd.merge(apps, df2, on = 'App', how = 'outer')

In [88]:
testing.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Last Updated,Review,Sentiment Category,Sentiment Score
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159.0,19.0,10000.0,Free,0.0,"January 7, 2018",NaN,NaN,NaN
1,Coloring book moana,ART_AND_DESIGN,3.9,967.0,14.0,500000.0,Free,0.0,"January 15, 2018",A kid's excessive ads. The types ads allowed a...,Negative,-0.250
2,Coloring book moana,ART_AND_DESIGN,3.9,967.0,14.0,500000.0,Free,0.0,"January 15, 2018",It bad >:(,Negative,-0.725
3,Coloring book moana,ART_AND_DESIGN,3.9,967.0,14.0,500000.0,Free,0.0,"January 15, 2018",like,Neutral,0.000
4,Coloring book moana,ART_AND_DESIGN,3.9,967.0,14.0,500000.0,Free,0.0,"January 15, 2018",NaN,NaN,NaN


In [89]:
top_10_user_feedback = testing[testing.Category == 'FINANCE']

In [90]:
top_10_user_feedback.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Last Updated,Review,Sentiment Category,Sentiment Score
14710,K PLUS,FINANCE,4.4,124424.0,NaN,10000000.0,Free,0.0,"June 26, 2018",NaN,NaN,NaN
14711,ING Banking,FINANCE,4.4,39041.0,NaN,1000000.0,Free,0.0,"August 3, 2018",NaN,NaN,NaN
14712,Citibanamex Movil,FINANCE,3.6,52306.0,42.0,5000000.0,Free,0.0,"July 27, 2018","Forget paying app, designed make fail payments...",Negative,-0.50
14713,Citibanamex Movil,FINANCE,3.6,52306.0,42.0,5000000.0,Free,0.0,"July 27, 2018","It's working expected, talking best bank Mexic...",Positive,0.40
14714,Citibanamex Movil,FINANCE,3.6,52306.0,42.0,5000000.0,Free,0.0,"July 27, 2018",It has many problems with Android 8.1. You can...,Positive,0.25


In [91]:
top_10_user_feedback = top_10_user_feedback[top_10_user_feedback.Type == 'Free']

In [92]:
top_10_user_feedback.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Last Updated,Review,Sentiment Category,Sentiment Score
14710,K PLUS,FINANCE,4.4,124424.0,NaN,10000000.0,Free,0.0,"June 26, 2018",NaN,NaN,NaN
14711,ING Banking,FINANCE,4.4,39041.0,NaN,1000000.0,Free,0.0,"August 3, 2018",NaN,NaN,NaN
14712,Citibanamex Movil,FINANCE,3.6,52306.0,42.0,5000000.0,Free,0.0,"July 27, 2018","Forget paying app, designed make fail payments...",Negative,-0.50
14713,Citibanamex Movil,FINANCE,3.6,52306.0,42.0,5000000.0,Free,0.0,"July 27, 2018","It's working expected, talking best bank Mexic...",Positive,0.40
14714,Citibanamex Movil,FINANCE,3.6,52306.0,42.0,5000000.0,Free,0.0,"July 27, 2018",It has many problems with Android 8.1. You can...,Positive,0.25


In [93]:
top_10_user_feedback = top_10_user_feedback.groupby('App')['Sentiment Score'].mean()

In [94]:
top_10_user_feedback = top_10_user_feedback.dropna()

In [95]:
top_10_user_feedback = top_10_user_feedback.sort_values(ascending = False)

In [96]:
type(top_10_user_feedback)

pandas.core.series.Series

In [97]:
top_10_user_feedback.to_frame()

,Sentiment Score
App,
BBVA Spain,0.515086
Associated Credit Union Mobile,0.388093
BankMobile Vibe App,0.353455
A+ Mobile,0.329592
Current debit card and app made for teens,0.327258
BZWBK24 mobile,0.326883
"Even - organize your money, get paid early",0.283929
Credit Karma,0.270052
Fortune City - A Finance App,0.266966


In [98]:
top_10_user_feedback = top_10_user_feedback.reset_index().head(10)

In [99]:
top_10_user_feedback.shape

(10, 2)